In [2]:
import pandas as pd
from simtk import unit as simtk_unit
from openff.toolkit.topology import Molecule, Topology
from openff.units import unit

from openff.system.stubs import ForceField
from openff.system.tests.energy_tests.openmm import (
    get_openmm_energies,
    _get_openmm_energies,
)
from openff.system.tests.energy_tests.gromacs import get_gromacs_energies

In [3]:
SMILES = "c1n(CCO)c(C(F)(F)(F))cc1CNCCl"

In [4]:
molecule = Molecule.from_smiles(SMILES)
molecule.generate_conformers(n_conformers=20, rms_cutoff=0.1 * simtk_unit.angstrom)
topology = molecule.to_topology()

In [5]:
parsley = ForceField("openff-1.1.0.offxml")

In [6]:
openff_sys = parsley.create_openff_system(topology)
openmm_sys = parsley.create_openmm_system(topology)

In [7]:
openff_sys.positions = molecule.conformers[0]
openff_sys.box = [4, 4, 4]

In [9]:
df = pd.DataFrame()

kj_mol = unit.kilojoule / unit.mol

for idx, conformer in enumerate(molecule.conformers):
    openff_sys.positions = conformer

    toolkit = sum(
        _get_openmm_energies(
            omm_sys=openmm_sys,
            box_vectors=openff_sys.box,
            positions=openff_sys.positions,
        ).energies.values()
    ).m_as(kj_mol)

    omm = sum(get_openmm_energies(openff_sys).energies.values()).m_as(kj_mol)

    gmx = sum(get_gromacs_energies(openff_sys).energies.values()).m_as(kj_mol)

    df = df.append(
        pd.DataFrame.from_dict(
            {
                "Conformer No.": [idx],
                "Toolkit (kJ/mol)": [round(toolkit, 3)],
                "System -> OpenMM (kJ/mol)": [round(omm, 3)],
                "System -> GROMACS": [round(gmx, 3)],
            }
        ),
        ignore_index=True,
    )

In [10]:
df.style.hide_index()

Conformer No.,Toolkit (kJ/mol),System -> OpenMM (kJ/mol),System -> GROMACS
0,333.472000,333.446000,335.346000
1,393.354000,393.332000,384.924000
2,323.614000,323.600000,332.331000
3,385.330000,385.269000,376.187000
4,333.148000,333.127000,342.992000
5,350.489000,350.463000,357.233000
6,454.499000,454.460000,460.799000
7,336.369000,336.337000,337.335000
8,413.470000,413.453000,407.706000
9,394.129000,394.073000,385.648000
